In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from datasets import load_dataset

# Load the 'google/fleurs' dataset for 'hi_in' (Hindi - India)
dset = load_dataset("SEACrowd/indo_general_mt_en_id", trust_remote_code=True)


In [2]:
import torch
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
checkpoint = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for memory efficiency
    trust_remote_code=True,
    device_map="auto"  # Automatically map the model to available GPUs
)

2024-12-15 21:23:08.372541: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 21:23:08.758218: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 21:23:08.758300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 21:23:08.829435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 21:23:08.965358: I tensorflow/core/platform/cpu_feature_guar

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
# Split the dataset into training, validation, and test sets
train_data = dset['train']
val_data = dset['validation']
test_data = dset['test']

In [4]:
pip install flash-attn


Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import csv
import numpy as np
import os
from nltk.translate.bleu_score import sentence_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 2
translations = []
bleu_scores = []

# Open file once
key=356
    # Process in batches
for i in range(356, len(test_data['src']), batch_size):
    batch_inputs = test_data['src'][i:i + batch_size]
    batch_references = test_data['tgt'][i:i + batch_size]

    # Create prompts
    # prompt_template = "Translate the following sentence from English to Indonesian:\nEnglish: {input_text}\nIndonesian:\n#answer\n"
    prompt_template = """
Translate the following sentences from English to Indonesian:

English: I am going to the market.
Indonesian: Saya akan pergi ke pasar.

English: What are you doing?
Indonesian: Apa yang sedang kamu lakukan?

English: This is my book.
Indonesian: Ini adalah buku saya.

English: {input_text}
Indonesian:
#answer
"""
    batch_prompts = [prompt_template.format(input_text=input_text) for input_text in batch_inputs]

    # Tokenize
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True).to(device)

    # Generate translations
    generate_ids = model.generate(inputs['input_ids'], max_length=300)
    batch_outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    # Extract translations and calculate BLEU
    for idx, (input_text, output_text_real, output_text) in enumerate(zip(batch_inputs, batch_references, batch_outputs)):

        # Extract the sentence one line below `#answer`
        translated_text = ""
        if "#answer" in output_text:
            # Split by lines and find the text after '#answer'
            lines = output_text.split("\n")
            try:
                index = lines.index("#answer") + 1  # Get the line after '#answer'
                translated_text = lines[index].strip()  # Clean up any extra spaces
                translated_text = translated_text.replace("Indonesian:", "").strip()
            except IndexError:
                translated_text = "Translation not found."
        else:
            translated_text = "No '#answer' marker in output."
        # Calculate BLEU score for this sentence
        bleu_score = sentence_bleu([output_text_real], translated_text)
        with open('predictions_phi35_test.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            if not os.path.exists('predictions_phi35_test.csv'):
                writer.writerow(['Index', 'Input', 'Predicted Output', 'Correct Output', 'BLEU Score'])
            writer.writerow([key + 1, input_text, translated_text, output_text_real, bleu_score])
        # Print the results
        print(f"Input: {input_text}")
        print(f"Predicted: {translated_text}")
        print(f"Reference: {output_text_real}")
        print(f"BLEU Score: {bleu_score:.4f}")
        print("-" * 50)
        bleu_scores.append(bleu_score)
        key +=1

# Print average BLEU score
print(f"Average BLEU Score: {np.mean(bleu_scores):.4f}")


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


Input: "The 2007 Baku terrorist plot was a foiled Islamic terrorist plot aimed at several targets in Baku, Azerbaijan."
Predicted: "Peluang terrorisme 2007 Baku adalah plot terrorisme Islam yang mengalami penghalangan yang berhubungan dengan tujuan sekitar beberapa target di Baku, Azerbaijan."
Reference: "Rencana teroris Baku 2007 adalah rencana teroris Islamik yang mengincar beberapa target di Baku, Azerbaijan."
BLEU Score: 0.5099
--------------------------------------------------
Input: "Many Muslims—and Orientalists—will tell you otherwise, of course, but the fact is that a fifth of the Koranic text is just incomprehensible."
Predicted: "Banyak muslim—dan orientalist—lahulat bahwa itu berarti lainnya, tapi fakta adalah bahwa sepertiga teks Koranan itu tidak bisa dipelajari."
Reference: "Banyak orang Muslim yang akan memberitahu anda sebaliknya, tentu saja, tetapi faktanya seperlima bagian dari teks Qur'an tidak dapat dimengerti."
BLEU Score: 0.4425
----------------------------------

/home/fachri/nusax/.conda/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Input: "Ethyl iodide (also iodoethane) is a colorless, flammable chemical compound."
Predicted: "Etil iodida (selain iodetan) adalah komposisi kimia warna kecil yang bersulur-bersulur."
Reference: Etil iodida (dikenal juga sebagai iodoetana) adalah senyawa kimia yang tak berwarna dan mudah terbakar.
BLEU Score: 0.4422
--------------------------------------------------
Input: "Wind: 4 mph, in from CF."
Predicted: "Angin: 4 mph, dari CF."
Reference: Pandangan seluas 10 derajat - tampilan melalui CRT.
BLEU Score: 0.0000
--------------------------------------------------
Input: "They hold them underwater, occasionally tilting them up for air."
Predicted: Mereka memeluknya di bawah air, sering-sering kayu kembangnya untuk mendapatkan udara.
Reference: "Mereka hidup di bawah air dan kadang-kadang muncul ke permukaan, tetapi hanya jika diprovokasi."
BLEU Score: 0.3454
--------------------------------------------------
Input: "By the end of the day, the coup attempt had failed in Jakarta at le

/home/fachri/nusax/.conda/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/fachri/nusax/.conda/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Input: Suture rather simple.
Predicted: This is my book.
Reference: Rasanya cukup ringan.
BLEU Score: 0.0000
--------------------------------------------------
Input: "Time To Know is an Israeli education technology program, that started as a philanthropic project in Israel in 2005, by Shmuel Meitar and Morris Kahn from the Aurec Group, who were the founders of Amdocs, a software company that specializes in customer care and billing systems for the telecom industry."
Predicted: "Time To Know adalah program teknologi pendidikan Israel, yang dimulainya pada tahun 2005, oleh Shmuel Meitar dan Morris Kahn dari Aurec Group. Mereka adalah pemimpin dari Amdocs, perusahaan software yang fokus pada sistem pelayanan pelanggan dan sistem pembayaran telekomunikasi."
Reference: "Ide untuk menyelenggarakan sebuah festival film internasional di Jakarta berasal dari pencinta film, Shanty Harmayn dan Natacha Devillers, yang pada 1999 bekerja sama di Salto Films dan bermukim di Jakarta."
BLEU Score: 0.2

KeyboardInterrupt: 

In [ ]:
# import torch
# import csv
# import numpy as np
# import os
# from nltk.translate.bleu_score import sentence_bleu
# # Check if CUDA (GPU) is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # List to store translations
# translations = []
# bleu_scores = []
# # Process each sentence in the test dataset
# for key, (input_text, output_text_real) in enumerate(zip(test_data['src'], test_data['tgt'])):
#     # Create the prompt for the current sentence
#     prompt = f"""
# Translate the following sentences from English to Indonesian:

# English: I am going to the market.
# Indonesian: Saya akan pergi ke pasar.

# English: {input_text}
# Indonesian:
# #answer
# """
#     # Tokenize the prompt
#     inputs = inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True).to(device)

#     # Generate the translation
#     generate_ids = model.generate(inputs['input_ids'], max_length=500)
    
#     # Decode the output and extract the translation
#     output_text = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
#     # Extract the sentence one line below `#answer`
#     translated_text = ""
#     if "#answer" in output_text:
#         # Split by lines and find the text after '#answer'
#         lines = output_text.split("\n")
#         try:
#             index = lines.index("#answer") + 1  # Get the line after '#answer'
#             translated_text = lines[index].strip()  # Clean up any extra spaces
#             translated_text = translated_text.replace("Indonesian:", "").strip()
#         except IndexError:
#             translated_text = "Translation not found."
#     else:
#         translated_text = "No '#answer' marker in output."
#     # Calculate BLEU score for this sentence
#     bleu_score = sentence_bleu([output_text_real], translated_text)
#     with open('predictions_phi35_test.csv', mode='a', newline='') as file:
#         if not os.path.exists('predictions_phi35_test.csv'):
#             writer.writerow(['Index', 'Input', 'Predicted Output', 'Correct Output', 'BLEU Score'])
#         writer = csv.writer(file)
#         writer.writerow([key + 1, input_text, translated_text, output_text_real, bleu_score])
#     # Print the results
#     print(f"Input: {input_text}")
#     print(f"Predicted: {translated_text}")
#     print(f"Reference: {output_text_real}")
#     print(f"BLEU Score: {bleu_score:.4f}")
#     print("-" * 50)
#     bleu_scores.append(bleu_score)
# print(np.mean(bleu_scores))

Input: Indira Gandhi imposed emergency rule in India.
Predicted: Indira Gandhi memperintah kebebasan operasi di India.
Reference: Kemudian Thoha Hanafi mendirikan perusahaan ekspor impor di Rengat.
BLEU Score: 0.1928
--------------------------------------------------
Input: "After a brief fight, the North Korean minelayer was sunk with a loss of all crew and no South Korean casualties were reported."
Predicted: "Setelah perang singkat, minelayer North Korea diperintahkan menghancurkan dengan kegagalan semua anggota dan tidak ada kasus kematian South Korea diperiksa."
Reference: "Setelah pertempuran singkat, kapal ranjau Korea Utara tenggelam dengan hilangnya semua awak dan tidak ada korban yang dilaporkan dari pihak Korea Selatan."
BLEU Score: 0.5412
--------------------------------------------------
Input: "Promoting culture as distinct to other goods and services, mapping and measuring the sectors adequately."
Predicted: "Mempromosikan budaya sebagai bagian yang unik dari barang dan 

/home/fachri/nusax/.conda/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Input: Most people here are of religious nature.
Predicted: Paling banyak orang di sini memiliki benda-benda religius.
Reference: Sebagian besar penduduknya beragama Islam.
BLEU Score: 0.0000
--------------------------------------------------
Input: "D. V-VIII; Gregory-Aland no. A or 02, Soden δ 4) is a fifth-century manuscript of the Greek Bible, containing the majority of the Septuagint and the New Testament."
Predicted: "D. V-VIII; Gregory-Aland no. A vau 02, Soden δ 4) adalah manusikra gereja yang berasal dari abad ke-VIII, yang mengandung sebagian besar Septuagint dan Teologi Baru."
Reference: "D. V-VIII; Gregory-Aland no. A or 02, Von Soden δ 4) adalah naskah Alkitab bahasa Yunani dari abad ke-5 M, memuat sebagian besar dari Perjanjian Lama (PL) versi Septuaginta, yaitu terjemahan bahasa Yunani dari Alkitab Ibrani, dan Perjanjian Baru (PB)."
BLEU Score: 0.4039
--------------------------------------------------
Input: """Everybody wanted to buy something for his or her family,"" s

/home/fachri/nusax/.conda/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Input: They should be impatient.
Predicted: Mereka seharusnya nyobolan.
Reference: Rakyat merasa tidak sabar menunggu.
BLEU Score: 0.0000
--------------------------------------------------
Input: "As the young king's senior tutor, Buchanan subjected James to regular beatings but also instilled in him a lifelong passion for literature and learning."
Predicted: "Buchanan, tutor senior dari reksa muda, mengusung James secara rutin penggandaan tetapi juga mendorong pada dia rasa takberuntung untuk keahlian sastra dan belajar-belajar."
Reference: "Sebagai guru senior raja muda, Buchanan menjadikan James sebagai bantingan biasa tetapi juga menanamkan hasrat seumur hidup untuk sastra dan pembelajaran."
BLEU Score: 0.5104
--------------------------------------------------
Input: "A ""living language"" is simply one that is used as the primary form of communication of a group of people."
Predicted: "Bahasa hidup adalah jenis bahasa yang hanya digunakan sebagai bentuk komunikasi utama oleh kelom

/home/fachri/nusax/.conda/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Input: """Rıza Doğan""."
Predicted: "Rıza Doğan."
Reference: Lahir dari Rahim’’.
BLEU Score: 0.0000
--------------------------------------------------
Input: "In 1984, Yayasan Menara Bhakti (Menara Bhakti Foundation) built the Menara Bhakti Campus."
Predicted: "Pada tahun 1984, Yayasan Menara Bhakti (Menara Bhakti Stb.) membangun Campus Menara Bhakti.
Reference: Tahun 1984 Yayasan Menara Bhakti berhasil membangun sebuah kampus yang diberi nama Kampus Menara Bhakti.
BLEU Score: 0.5929
--------------------------------------------------
Input: While doing Analysis we are trying to understand the problem.
Predicted: Sedang melakukan analisis, kami berusaha memahami masalahnya.
Reference: Peristiwa itu kemudian kita analisis untuk mencari penyebabnya.
BLEU Score: 0.2626
--------------------------------------------------
Input: Attempts by the government to sustain the industry by introducing compulsory crop spraying after 1890 failed.
Predicted: Percobaan oleh pemerintah untuk melestarikan 

KeyboardInterrupt: 